In [1]:
import time
import datetime
import torch
import torch.nn.functional as Encoder
from torchvision import datasets
from torch.optim import SGD
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt

## ANN Model & Training Method 

In [4]:
class ConfigDataset(Dataset):
    def __init__(self, path):
        """X is the training input while Y is the target output."""

        # Loading PyTorch dataset,
        self.X, self.Y = torch.load(path)

        # Normalisation of pixel intensity values,
        self.X = self.X / 255
        self.X = self.X.view(-1, 28**2)

        # One-hot encoding the target output,
        self.Y = Y_encoded = Encoder.one_hot(self.Y, num_classes = 10).to(float)
        
    def __len__(self):
        """Returns the number of objects in the dataset."""
        return self.X.shape[0]
    
    def __getitem__(self, index):
        "Returns a tuple of training input and target output."
        return self.X[index], self.Y[index]

class NeuralNetwork(torch.nn.Module):
    """The neutral network architecture."""

    def __init__(self):
        """Creating the layer stucture and activation functions of the neutral network."""
        super().__init__()

        # Layers,
        self.DenseLayer_INPUT = torch.nn.Linear(784, 20)
        self.DenseLayer_HIDDEN_1 = torch.nn.Linear(20, 20)
        self.DenseLayer_HIDDEN_2 = torch.nn.Linear(20, 20)
        self.DenseLayer_OUTPUT = torch.nn.Linear(20, 10)

        # Activation functions,
        self.ReLU = torch.nn.ReLU()
        self.SoftMax = torch.nn.Softmax()

    def forward(self, X):
        """Defining the forward propagation."""
        X = self.DenseLayer_INPUT(X)
        X = self.ReLU(self.DenseLayer_HIDDEN_1(X))
        X = self.ReLU(self.DenseLayer_HIDDEN_2(X))
        X = self.SoftMax(self.DenseLayer_OUTPUT(X))

        return X

def TrainModel(training_data, neural_network, n_epochs = 10):

    # Utilisation of CUDA if possible,
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print(f'CUDA is available. Using GPU: {torch.cuda.get_device_name(0)}')
    else:
        device = torch.device("cpu")
        print('CUDA is not available. Using CPU.')

    neural_network.to(device)

    # Stochastic Gradient Descent (SGD) as optimiser,
    optimiser = SGD(neural_network.parameters(), lr = 0.01)

    # Cross-Entropy loss function,
    loss_function = torch.nn.CrossEntropyLoss()

    losses = []
    epochs = np.arange(start = 1, stop = n_epochs, step = 1)

    for epoch in range(1, n_epochs + 1):
        epoch_loss = []
        stopwatch_start = time.time()
        for i, (X, Y) in enumerate(training_data):

            # Move data to selected device,
            X, Y = X.to(device), Y.to(device)

            # Back propagation,
            optimiser.zero_grad() 
            loss_value = loss_function(neural_network(X), Y) 
            epoch_loss.append(loss_value.item())
            loss_value.backward(loss_value) 
            optimiser.step()

        stopwatch_stop = time.time()
        epoch_time = round(stopwatch_stop - stopwatch_start, 2)
        epoch_avgloss = np.mean(epoch_loss)
        losses.append(epoch_avgloss)
        ETA = str(datetime.timedelta(seconds = (n_epochs - epoch)*epoch_time)).split(".")[0]

        print(f'[Completed Epoch: {epoch}/{n_epochs} ︱ Time Taken: {epoch_time} secs ︱ Loss: {epoch_avgloss:.5f} ︱ ETA: {ETA} ]')

    torch.save(neural_network.state_dict(), "model.pth")
    return np.array(epochs), np.array(losses)

def AssessModel(test_data, neural_network):

    X_test = test_data[:][0]
    Y_test = test_data[:][1]
    Yhat_test = neural_network(X_test)

    correct_counter = 0
    incorrect_counter = 0
    for Y, Yhat in zip(Y_test, Yhat_test):
        if Yhat.argmax() == Y.argmax():
            correct_counter += 1
        else:
            incorrect_counter += 1

    accuracy = correct_counter/(correct_counter + incorrect_counter)

    return accuracy

## Training Algorithm

In [6]:
# Creating and training neural network,
TRAINING_DATASET = ConfigDataset("training.pt")
TRAINING_DATASET_LOADED = DataLoader(TRAINING_DATASET, batch_size = 5)
NEURAL_NETWORK = NeuralNetwork()
EPOCH_ARRAY, LOSS_ARRAY = TrainModel(TRAINING_DATASET_LOADED, NEURAL_NETWORK, n_epochs = 25)

# Testing neural network,
TEST_DATASET = ConfigDataset("test.pt")
print("Accuracy: " + str(AssessModel(TEST_DATASET, NEURAL_NETWORK)))

CUDA is not available. Using CPU.


C:\Users\Prithvi\AppData\Local\Temp\ipykernel_16660\1536117356.py:45: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  X = self.SoftMax(self.DenseLayer_OUTPUT(X))


[Completed Epoch: 1/25 ︱ Time Taken: 9.16 secs ︱ Loss: 1.93788 ︱ ETA: 0:03:39 ]
[Completed Epoch: 2/25 ︱ Time Taken: 10.22 secs ︱ Loss: 1.59070 ︱ ETA: 0:03:55 ]
[Completed Epoch: 3/25 ︱ Time Taken: 8.62 secs ︱ Loss: 1.57064 ︱ ETA: 0:03:09 ]
[Completed Epoch: 4/25 ︱ Time Taken: 8.74 secs ︱ Loss: 1.55992 ︱ ETA: 0:03:03 ]
[Completed Epoch: 5/25 ︱ Time Taken: 11.72 secs ︱ Loss: 1.55423 ︱ ETA: 0:03:54 ]
[Completed Epoch: 6/25 ︱ Time Taken: 9.82 secs ︱ Loss: 1.55042 ︱ ETA: 0:03:06 ]
[Completed Epoch: 7/25 ︱ Time Taken: 10.81 secs ︱ Loss: 1.54680 ︱ ETA: 0:03:14 ]
[Completed Epoch: 8/25 ︱ Time Taken: 9.38 secs ︱ Loss: 1.54415 ︱ ETA: 0:02:39 ]
[Completed Epoch: 9/25 ︱ Time Taken: 10.91 secs ︱ Loss: 1.54072 ︱ ETA: 0:02:54 ]
[Completed Epoch: 10/25 ︱ Time Taken: 10.39 secs ︱ Loss: 1.53865 ︱ ETA: 0:02:35 ]
[Completed Epoch: 11/25 ︱ Time Taken: 11.33 secs ︱ Loss: 1.53712 ︱ ETA: 0:02:38 ]
[Completed Epoch: 12/25 ︱ Time Taken: 11.83 secs ︱ Loss: 1.53478 ︱ ETA: 0:02:33 ]
[Completed Epoch: 13/25 ︱ Time